In [8]:
import os
import pandas as pd
from fuzzywuzzy import fuzz
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import requests 
import numpy as np
from geopy.exc import GeocoderTimedOut

from requests.structures import CaseInsensitiveDict
from sklearn.linear_model import LinearRegression, RANSACRegressor

In [18]:
filename = "myAllData.csv"
df = pd.read_csv(filename)
A_cleaned = [location for location in df["location1"] if not(pd.isnull(location))]
B_cleaned = [location for location in df["location1"].unique() if not(pd.isnull(location))]


tuples_list = [max([(fuzz.token_set_ratio(i,j),j) for j in B_cleaned]) for i in A_cleaned]
similarity_score, fuzzy_match = map(list,zip(*tuples_list))
df2 = pd.DataFrame({ "location1": fuzzy_match})
df['location1'] = df2[df2['location1'].isin(df2['location1'])]['location1'].values



A_cleaned = [location for location in df["location2"] if not(pd.isnull(location))]
B_cleaned = [location for location in df["location2"].unique() if not(pd.isnull(location))]
tuples_list = [max([(fuzz.token_set_ratio(i,j),j) for j in B_cleaned]) for i in A_cleaned]
similarity_score, fuzzy_match = map(list,zip(*tuples_list))
df2 = pd.DataFrame({ "location2": fuzzy_match})
df['location2'] = df2[df2['location2'].isin(df2['location2'])]['location2'].values
#df.to_csv('simi.csv',encoding='UTF-8-sig')

In [21]:
Country = "مصر"
for i_city in df['city'].unique():
    df2 = df.loc[df['city'] == i_city]
    for i_location in (df2['location1'].unique()):
     # num =  df.at[i,'bedrooms'].split():

        try:
            url = f"https://api.geoapify.com/v1/geocode/search?&city={i_location} {i_city}&country={Country}&lang=ar&limit=1&filter=countrycode:eg&format=json&apiKey=9603e9617cf64f41b62092fda88ce902"
            headers = CaseInsensitiveDict()
            headers["Accept"] = "application/json"

            resp = requests.get(url, headers=headers)
            lon = resp.json()['results'][0]['lon']
            lat = resp.json()['results'][0]['lat']
           
        except:

            lon = np.nan
            lat = np.nan
        df.loc[df['location1'] == i_location, 'lon'] = lon
        df.loc[df['location1'] == i_location, 'lat'] = lat

        # # print(i_location, resp.json()['results'][0]
        # ['lon'], resp.json()['results'][0]['lat'])

#df.to_csv('Cairo_Data.csv',encoding='UTF-8-sig')

In [24]:
df['outlier'] = np.nan
c = 0
ex = 0
for i_city in df['city'].unique():
    for i_rent_sale in df['rent_sale'].unique():
        for i_location in df['location1'].unique():
            for i_unit in df['unit'].unique():
                try:
                    filtered = df.loc[(df['unit'] == i_unit) & (df['location1'] == i_location) &
                                      (df['rent_sale'] == i_rent_sale) & (df['city'] == i_city)]
                    # print(len(filtered))
                    if (len(filtered) > 1):
                        x = (filtered['size'].values.reshape(-1, 1))
                        y = (filtered['price'].values.reshape(-1, 1))
                        t = 1.96 * filtered['price'].std()
                        ransac = RANSACRegressor(base_estimator=LinearRegression(),
                                                 min_samples=2, max_trials=3,
                                                 residual_threshold=t, random_state=42)
                        ransac.fit(x, y)
                        outlier_mask = np.logical_not(ransac.inlier_mask_)
                        df.loc[filtered.index, ['outlier']] = outlier_mask
                        c += 1
                except:
                    ex += 1
                    pass



df['outlier'].fillna(False, inplace=True)